In [5]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [6]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [7]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [8]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [9]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [10]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.734 seconds.
Prefix dict has been built succesfully.


In [11]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [12]:
from random import shuffle
word_data[8]

['货到付款',
 '一步',
 '赢',
 '2014',
 '秋季',
 '松糕',
 '帆布鞋',
 '女',
 '韩版潮',
 '厚底',
 '松糕鞋',
 '学生',
 '高帮',
 '休闲',
 '板鞋',
 '黑',
 '白色',
 '35',
 '正规',
 '码']

## 标签分离(一个标签分成三个标签)

In [13]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [14]:
len(data_y["label1"].unique())

22

In [15]:
len(data_y["label2"].unique())

191

In [16]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [17]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [18]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

# 词向量训练

In [19]:
from gensim.models.word2vec import Word2Vec 
if os.path.exists("w2v_model"):
    model = Word2Vec.load("w2v_model")
    print(1)
else:
    sentences = word_data
    model= Word2Vec(size=50, window=10, min_count = 1)
    model.build_vocab(sentences)
    model.train(sentences,total_examples = model.corpus_count,epochs = model.iter)
    model.save("w2v_model")

1


## 填充词向量(构造输入输出)（让每个句子拥有同样的维度）

In [20]:
from keras.preprocessing import sequence
x_train = [model[word] for word in word_data_train]
x_test = [model[word] for word in word_data_test]
x_train = sequence.pad_sequences(x_train, maxlen=18)
x_test  = sequence.pad_sequences(x_test, maxlen=18) 

c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
c:\users\richado\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


## 默认参数


## TextCnn

In [66]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, maxlen, sequence_length, num_classes, filter_sizes, embedding_size, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.float32, [None, sequence_length, embedding_size], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        self.hidden_size = 256
        self.n_layer = 1
        self.batch_size = 256
        
        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)
        self.keep_prob = 0.9
        pooled_outputs = []
        self.input_x_expand = tf.expand_dims(self.input_x, -1)
        print(self.input_x_expand.shape)
        with tf.variable_scope('bigru_text'):
            self.gru_output = self.bigru_inference(self.input_x)
            print("gru shape", self.gru_output.shape)
#         for i, filter_size in enumerate(filter_sizes):
#             with tf.name_scope("conv-maxpool-%s" % filter_size):
#                 # Convolution Layer
#                 filter_shape = [filter_size, embedding_size, 1, num_filters]
#                 W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
#                 b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
#                 # 卷积
#                 conv = tf.nn.conv2d(
#                     self.input_x_expand,
#                     W,
#                     strides=[1, 1, 1, 1],
#                     padding="VALID",
#                     name="conv")
        
        
#                 h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
            
#                 # 最大池化
#                 pooled = tf.nn.max_pool(
#                     h,
#                     ksize=[1, sequence_length - filter_size + 1, 1, 1],
#                     strides=[1, 1, 1, 1],
#                     padding='VALID',
#                     name="pool")
#                 pooled = tf.nn.dropout(pooled, keep_prob)
#                 pooled_outputs.append(pooled)
                

#         # Combine all the pooled features
#         num_filters_total = num_filters * len(filter_sizes)
#         self.h_pool = tf.concat(pooled_outputs, 3)
#         print(self.h_pool.shape)
        
#         self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])
#         self.h_drop = self.h_pool_flat
        
        # Final (unnormalized) scores and predictions
        self.h_drop = self.gru_output
        num_filters_total = self.h_drop.shape[1]
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)
            
    def gru_cell(self):
        with tf.name_scope('gru_cell'):
            cell = rnn.GRUCell(self.hidden_size, reuse=tf.get_variable_scope().reuse)
        return rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

    def bi_gru(self, inputs):
        """build the bi-GRU network. 返回个所有层的隐含状态。"""
        cells_fw = [self.gru_cell() for _ in range(self.n_layer)]
        cells_bw = [self.gru_cell() for _ in range(self.n_layer)]
        initial_states_fw = [cell_fw.zero_state(self.batch_size, tf.float32) for cell_fw in cells_fw]
        initial_states_bw = [cell_bw.zero_state(self.batch_size, tf.float32) for cell_bw in cells_bw]
        outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs,
                                                            initial_states_fw=initial_states_fw,
                                                            initial_states_bw=initial_states_bw, dtype=tf.float32)
        return outputs
    
    def bigru_inference(self, X_inputs):
#         inputs = tf.nn.embedding_lookup(self.title_embedding, X_inputs)
        output_bigru = self.bi_gru(X_inputs)
        output_att = self.task_specific_attention(output_bigru, self.hidden_size*2)
        return output_att
    
    def task_specific_attention(self, inputs, output_size,
                                initializer=layers.xavier_initializer(),
                                activation_fn=tf.tanh, scope=None):
        """
        Performs task-specific attention reduction, using learned
        attention context vector (constant within task of interest).
        Args:
            inputs: Tensor of shape [batch_size, units, input_size]
                `input_size` must be static (known)
                `units` axis will be attended over (reduced from output)
                `batch_size` will be preserved
            output_size: Size of output's inner (feature) dimension
        Returns:
           outputs: Tensor of shape [batch_size, output_dim].
        """
#         assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
        with tf.variable_scope(scope or 'attention') as scope:
            # u_w, attention 向量
            attention_context_vector = tf.get_variable(name='attention_context_vector', shape=[output_size],
                                                       initializer=initializer, dtype=tf.float32)
            # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
            input_projection = layers.fully_connected(inputs, output_size, activation_fn=activation_fn, scope=scope)
            # 输出 [batch_size, units]
            vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
            attention_weights = tf.nn.softmax(vector_attn, dim=1)
            tf.summary.histogram('attention_weigths', attention_weights)
            weighted_projection = tf.multiply(inputs, attention_weights)
            outputs = tf.reduce_sum(weighted_projection, axis=1)
            return outputs  # 输出 [batch_size, hidden_size*2]

In [67]:
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

In [72]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn

# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
maxlen = 16

batch_size = 256 

epochs = 100

is_training = True
# 词向量长度
embedding_dims = 50

filters = 64
kernel_size = 5
hidden_dims = 128
epochs = 100

# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")
# tf.flags.DEFINE_string("positive_data_file", "./data/rt-polaritydata/rt-polarity.pos", "Data source for the positive data.")
# tf.flags.DEFINE_string("negative_data_file", "./data/rt-polaritydata/rt-polarity.neg", "Data source for the negative data.")

# Model Hyperparameters
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
#           allow_soft_placement=FLAGS.allow_soft_placement,
#           log_device_placement=FLAGS.log_device_placement
        )
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(maxlen = 10, sequence_length = 18, num_classes = [22, 191, 1192], filter_sizes = [1,2, 3, 4, 5], embedding_size = 50, num_filters = 64)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3
                }

#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), batch_size, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev
                    }
                    if len(x_batch_dev) < batch_size:
                        continue
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                if len(x_batch) < batch_size:
                    continue
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [73]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)

(?, 18, 50, 1)
gru shape (256, 512)
2019-02-22T01:39:29.884731: step 40, loss 2.44305, acc [0.78710938 0.64316406 0.51787109 0.45302734]
2019-02-22T01:39:34.883778: step 80, loss 1.53607, acc [0.86201172 0.74980469 0.64560547 0.5828125 ]
2019-02-22T01:39:39.629999: step 120, loss 1.38422, acc [0.85957031 0.76123047 0.6625     0.59833984]
2019-02-22T01:39:44.579535: step 160, loss 1.27128, acc [0.86259766 0.76982422 0.67626953 0.61035156]
2019-02-22T01:39:49.324760: step 200, loss 1.17051, acc [0.88134766 0.78798828 0.69638672 0.634375  ]

Evaluation:
2019-02-22T01:40:12.246401: step 200, loss 1.14838, acc [0.87030604 0.78310925 0.70642413 0.63931964] 

2019-02-22T01:40:19.285055: step 240, loss 1.15561, acc [0.87734375 0.78955078 0.70244141 0.64179688]
2019-02-22T01:40:25.801169: step 280, loss 1.13035, acc [0.87695312 0.79267578 0.70136719 0.64208984]
2019-02-22T01:40:32.329185: step 320, loss 1.10844, acc [0.87490234 0.79521484 0.71113281 0.64638672]
2019-02-22T01:40:38.531223: step 

TypeError: 'NoneType' object is not iterable

In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape